## Install Required Packages

In [ ]:
!pip install requests pandas


## Import Libraries

In [ ]:
import requests
import pandas as pd
from datetime import datetime

## API Configuration

In [ ]:
API_KEY = '1d32fa59494f31b5b5105d64c8d36498'
CITY = 'Jaipur'
BASE_URL = 'http://api.openweathermap.org/data/2.5/forecast'

## Make API Request

In [ ]:
params = {
    'q': CITY,
    'appid': API_KEY,
    'units': 'metric'
}

response = requests.get(BASE_URL, params=params)
data = response.json()

if response.status_code != 200:
    print(f"Error: {data.get('message', 'Unknown error')}")
    exit()

## Extract Data

In [ ]:
forecasts = data['list']

timestamps = []
temperatures = []
feels_like = []
humidity = []
wind_speed = []
weather_desc = []
rain_3h = []

for forecast in forecasts:
    timestamps.append(forecast['dt'])
    temperatures.append(forecast['main']['temp'])
    feels_like.append(forecast['main']['feels_like'])
    humidity.append(forecast['main']['humidity'])
    wind_speed.append(forecast['wind']['speed'])
    weather_desc.append(forecast['weather'][0]['description'])
    rain_3h.append(forecast.get('rain', {}).get('3h', 0))

## Create and Clean DataFrame

In [ ]:
weather_df = pd.DataFrame({
    'timestamp': timestamps,
    'temperature': temperatures,
    'feels_like': feels_like,
    'humidity': humidity,
    'wind_speed': wind_speed,
    'weather_description': weather_desc,
    'rain_3h': rain_3h
})

weather_df['datetime'] = pd.to_datetime(weather_df['timestamp'], unit='s')
weather_df.set_index('datetime', inplace=True)
weather_df.drop('timestamp', axis=1, inplace=True)

weather_df['rain_3h'] = weather_df['rain_3h'].fillna(0)
weather_df['weather_description'] = weather_df['weather_description'].str.lower().str.strip()

temp_mask = (weather_df['temperature'] < -50) | (weather_df['temperature'] > 50)
weather_df.loc[temp_mask, 'temperature'] = weather_df['temperature'].median()

## Summary

In [ ]:
print(f"Duplicate rows: {weather_df.duplicated().sum()}")
print("\nData Types:\n", weather_df.dtypes)
print("\nSummary:\n", weather_df.describe())
print("\nSample:\n", weather_df.head())

Duplicate rows: 0

Data Types:
 temperature            float64
feels_like             float64
humidity                 int64
wind_speed             float64
weather_description     object
rain_3h                  int64
dtype: object

Summary:
        temperature  feels_like   humidity  wind_speed  rain_3h
count    40.000000   40.000000  40.000000   40.000000     40.0
mean     36.551500   35.156000  22.550000    5.420250      0.0
std       3.930101    3.624348   7.472857    1.332925      0.0
min      30.140000   29.130000  11.000000    1.070000      0.0
25%      33.515000   32.235000  15.750000    4.640000      0.0
50%      36.545000   34.545000  22.000000    5.705000      0.0
75%      39.970000   38.415000  28.000000    6.482500      0.0
max      42.810000   40.700000  36.000000    7.240000      0.0

Sample:
                      temperature  feels_like  humidity  wind_speed  \
datetime                                                             
2025-05-14 09:00:00        40.15       3

Save Cleaned Data

In [ ]:
weather_df.to_csv('jaipur_weather_forecast_cleaned.csv')